**This notebook is an exercise in the [SQL](https://www.kaggle.com/learn/intro-to-sql) course.  You can reference the tutorial at [this link](https://www.kaggle.com/dansbecker/joining-data).**

---


# Introduction

[Stack Overflow](https://stackoverflow.com/) is a widely beloved question and answer site for technical questions. You'll probably use it yourself as you keep using SQL (or any programming language). 

Their data is publicly available. What cool things do you think it would be useful for?

Here's one idea:
You could set up a service that identifies the Stack Overflow users who have demonstrated expertise with a specific technology by answering related questions about it, so someone could hire those experts for in-depth help.

In this exercise, you'll write the SQL queries that might serve as the foundation for this type of service.

As usual, run the following cell to set up our feedback system before moving on.

In [1]:
# Set up feedback system
from learntools.core import binder
binder.bind(globals())
from learntools.sql.ex6 import *
print("Setup Complete")

Using Kaggle's public dataset BigQuery integration.
Setup Complete


Run the next cell to fetch the `stackoverflow` dataset.

In [2]:
from google.cloud import bigquery

# Create a "Client" object
client = bigquery.Client()

# Construct a reference to the "stackoverflow" dataset
dataset_ref = client.dataset("stackoverflow", project="bigquery-public-data")

# API request - fetch the dataset
dataset = client.get_dataset(dataset_ref)

Using Kaggle's public dataset BigQuery integration.


# Exercises

### 1) Explore the data

Before writing queries or **JOIN** clauses, you'll want to see what tables are available. 

*Hint*: Tab completion is helpful whenever you can't remember a command. Type `client.` and then hit the tab key. Don't forget the period before hitting tab.

In [8]:
# Get a list of available tables 
tables = list(client.list_tables(dataset))
list_of_tables = [table.table_id for table in tables] 

# Print your answer
print(list_of_tables)

# Check your answer
q_1.check()

['badges', 'comments', 'post_history', 'post_links', 'posts_answers', 'posts_moderator_nomination', 'posts_orphaned_tag_wiki', 'posts_privilege_wiki', 'posts_questions', 'posts_tag_wiki', 'posts_tag_wiki_excerpt', 'posts_wiki_placeholder', 'stackoverflow_posts', 'tags', 'users', 'votes']


<IPython.core.display.Javascript object>

<span style="color:#33cc33">Correct</span>

For the solution, uncomment the line below.

In [7]:
q_1.solution()

<IPython.core.display.Javascript object>

<span style="color:#33cc99">Solution:</span> 
```python

# Get a list of available tables 
tables = list(client.list_tables(dataset))
list_of_tables = [table.table_id for table in tables] 

```

### 2) Review relevant tables

If you are interested in people who answer questions on a given topic, the `posts_answers` table is a natural place to look. Run the following cell, and look at the output.

In [9]:
# Construct a reference to the "posts_answers" table
answers_table_ref = dataset_ref.table("posts_answers")

# API request - fetch the table
answers_table = client.get_table(answers_table_ref)

# Preview the first five lines of the "posts_answers" table
client.list_rows(answers_table, max_results=5).to_dataframe()

,id,title,body,accepted_answer_id,answer_count,comment_count,community_owned_date,creation_date,favorite_count,last_activity_date,last_edit_date,last_editor_display_name,last_editor_user_id,owner_display_name,owner_user_id,parent_id,post_type_id,score,tags,view_count
0,26292293,None,<p>You could use <code>NULL</code> as the de f...,None,None,0,None,2014-10-10 05:05:21.923000+00:00,None,2014-10-10 05:05:21.923000+00:00,NaT,None,NaN,None,1342984,26292258,2,0,None,None
1,26292297,None,<p>You should add a class for <code>img</code>...,None,None,0,None,2014-10-10 05:05:42.760000+00:00,None,2014-10-10 05:05:42.760000+00:00,NaT,None,NaN,None,3607246,26292233,2,0,None,None
2,26292304,None,<pre><code>sed 's/\.Precilla123/.Precill/g'\n<...,None,None,0,None,2014-10-10 05:07:16.467000+00:00,None,2014-10-10 05:07:16.467000+00:00,NaT,None,NaN,None,2885763,26291141,2,0,None,None
3,26292318,None,<p>You can use:</p>\n\n<pre><code>$('#header1'...,None,None,0,None,2014-10-10 05:08:20.230000+00:00,None,2014-10-10 05:23:27.557000+00:00,2014-10-10 05:23:27.557000+00:00,None,3558931.0,None,3558931,26292296,2,0,None,None
4,26292325,None,<pre><code>$('#header1').html('Year: &lt;b&gt;...,None,None,0,None,2014-10-10 05:08:46.490000+00:00,None,2014-10-10 05:08:46.490000+00:00,NaT,None,NaN,None,2929162,26292296,2,0,None,None


It isn't clear yet how to find users who answered questions on any given topic. But `posts_answers` has a `parent_id` column. If you are familiar with the Stack Overflow site, you might figure out that the `parent_id` is the question each post is answering.

Look at `posts_questions` using the cell below.

In [10]:
# Construct a reference to the "posts_questions" table
questions_table_ref = dataset_ref.table("posts_questions")

# API request - fetch the table
questions_table = client.get_table(questions_table_ref)

# Preview the first five lines of the "posts_questions" table
client.list_rows(questions_table, max_results=5).to_dataframe()

,id,title,body,accepted_answer_id,answer_count,comment_count,community_owned_date,creation_date,favorite_count,last_activity_date,last_edit_date,last_editor_display_name,last_editor_user_id,owner_display_name,owner_user_id,parent_id,post_type_id,score,tags,view_count
0,34379321,AspectJ: Issue on Weblogic 12,<p>I worked on a Java EE application with foll...,34395439.0,1,0,None,2015-12-20 08:50:30.557000+00:00,NaN,2015-12-25 12:42:56.390000+00:00,2015-12-25 12:42:56.390000+00:00,None,472792.0,None,1103179,None,1,0,java-8|aspectj|weblogic12c|aspectj-maven-plugin,256
1,34402037,how to make rgba in Stylus,"<p>Hi guys i'm new with Stylus, want to know h...",34402541.0,1,0,None,2015-12-21 18:23:43.557000+00:00,NaN,2015-12-21 18:58:13.680000+00:00,NaT,None,NaN,None,5531191,None,1,1,css|stylus,256
2,34450038,How to do (i.e. derive) multiplication (using ...,<p>I'm implementing a small program to handle ...,34453505.0,1,0,None,2015-12-24 09:02:07.160000+00:00,1.0,2015-12-26 19:05:55.280000+00:00,2015-12-24 15:25:18.033000+00:00,None,548515.0,None,4533142,None,1,1,haskell|typeclass,256
3,34455963,Different value displaying in Razor textbox th...,<p>I'm having an issue where a value I set in ...,34456010.0,1,0,None,2015-12-24 17:13:24.017000+00:00,NaN,2015-12-24 17:21:08.460000+00:00,NaT,None,NaN,None,1428743,None,1,0,asp.net|asp.net-mvc|asp.net-mvc-4|razor,256
4,34492056,Swfloader scale content to full size,<p>I have load flash swf file to my flex proje...,NaN,1,0,None,2015-12-28 10:46:53.820000+00:00,NaN,2015-12-29 07:59:31.167000+00:00,2015-12-29 07:59:31.167000+00:00,None,124084.0,None,1213296,None,1,0,actionscript-3|apache-flex|swfloader,256


Are there any fields that identify what topic or technology each question is about? If so, how could you find the IDs of users who answered questions about a specific topic?

Think about it, and then check the solution by running the code in the next cell.

In [11]:
# Check your answer (Run this code cell to receive credit!)
q_2.solution()

<IPython.core.display.Javascript object>

<span style="color:#33cc99">Solution:</span> 
`posts_questions` has a column called `tags` which lists the topics/technologies each question is about.

`posts_answers` has a column called `parent_id` which identifies the ID of the question each answer is responding to.
`posts_answers` also has an `owner_user_id` column which specifies the ID of the user who answered the question.

You can join these two tables to:
- determine the `tags` for each answer, and then
- select the `owner_user_id` of the answers on the desired tag.

This is exactly what you will do over the next few questions.


### 3) Selecting the right questions

A lot of this data is text. 

We'll explore one last technique in this course which you can apply to this text.

A **WHERE** clause can limit your results to rows with certain text using the **LIKE** feature. For example, to select just the third row of the `pets` table from the tutorial, we could use the query in the picture below.

![](https://i.imgur.com/RccsXBr.png) 

You can also use `%` as a "wildcard" for any number of characters. So you can also get the third row with:

```
query = """
        SELECT * 
        FROM `bigquery-public-data.pet_records.pets` 
        WHERE Name LIKE '%ipl%'
        """
```

Try this yourself. Write a query that selects the `id`, `title` and `owner_user_id` columns from the `posts_questions` table. 
- Restrict the results to rows that contain the word "bigquery" in the `tags` column. 
- Include rows where there is other text in addition to the word "bigquery" (e.g., if a row has a tag "bigquery-sql", your results should include that too).

In [14]:
# Your code here
questions_query = """
                  SELECT id, title , owner_user_id
                  FROM `bigquery-public-data.stackoverflow.posts_questions`
                  WHERE  tags  LIKE '%bigquery%'
                  """

# Set up the query (cancel the query if it would use too much of 
# your quota, with the limit set to 1 GB)
safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=10**10)
questions_query_job = client.query(questions_query, job_config=safe_config) # Your code goes here

# API request - run the query, and return a pandas DataFrame
questions_results = questions_query_job.to_dataframe() # Your code goes here

# Preview results
print(questions_results.head())

# Check your answer
q_3.check()

         id                                              title  owner_user_id
0  18838880  formatting timestamp from BigQuery to Google S...       132438.0
1  19024171  BigQuery OAuth access token on Java App Engine...        78212.0
2  18905506  Using Table Decorators on Big Query Web Interface      2723931.0
3  19119761  Google BigQuery: Are we charged when resources...      1401986.0
4  19037703  Missing parameters when creating new table in ...      1681143.0


<IPython.core.display.Javascript object>

<span style="color:#33cc33">Correct</span>

For a hint or the solution, uncomment the appropriate line below.

In [ ]:
#q_3.hint()
#q_3.solution()

### 4) Your first join
Now that you have a query to select questions on any given topic (in this case, you chose "bigquery"), you can find the answers to those questions with a **JOIN**.  

Write a query that returns the `id`, `body` and `owner_user_id` columns from the `posts_answers` table for answers to "bigquery"-related questions. 
- You should have one row in your results for each answer to a question that has "bigquery" in the tags.  
- Remember you can get the tags for a question from the `tags` column in the `posts_questions` table.

Here's a reminder of what a **JOIN** looked like in the tutorial:
```
query = """
        SELECT p.Name AS Pet_Name, o.Name AS Owner_Name
        FROM `bigquery-public-data.pet_records.pets` as p
        INNER JOIN `bigquery-public-data.pet_records.owners` as o 
            ON p.ID = o.Pet_ID
        """
```

It may be useful to scroll up and review the first several rows of the `posts_answers` and `posts_questions` tables.  

In [17]:
# Your code here
answers_query = """
                SELECT a.id, a.body, a.owner_user_id
                FROM `bigquery-public-data.stackoverflow.posts_questions` AS q 
                INNER JOIN `bigquery-public-data.stackoverflow.posts_answers` AS a
                    ON q.id = a.parent_id
                WHERE q.tags LIKE '%bigquery%'
                """

# Set up the query
safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=10**10)
answers_query_job = client.query(answers_query, job_config=safe_config) # Your code goes here

# API request - run the query, and return a pandas DataFrame
answers_results = answers_query_job.to_dataframe() # Your code goes here

# Preview results
print(answers_results.head())

# Check your answer
q_4.check()

         id                                               body  owner_user_id
0  59562506  <p>I finally figured out a way to extract data...      3759168.0
1  59564850  <p>I found a similar error <a href="https://cl...     11928099.0
2  59567653  <p>I found the answer. I needed to use names.e...      4326515.0
3  59571214  <p>When I tested your code, I received the fol...      1393015.0
4  59579909  <p>From your description looks like you are no...     11504909.0


<IPython.core.display.Javascript object>

<span style="color:#33cc33">Correct</span>

For a hint or the solution, uncomment the appropriate line below.

In [15]:
q_4.hint()
q_4.solution()

<IPython.core.display.Javascript object>

<span style="color:#3366cc">Hint:</span> 
Do an **INNER JOIN** between `bigquery-public-data.stackoverflow.posts_questions` and `bigquery-public-data.stackoverflow.posts_answers`.

Give `post_questions` an alias of `q`, and use `a` as an alias for `posts_answers`. The **ON** part of your join is `q.id = a.parent_id`.


<IPython.core.display.Javascript object>

<span style="color:#33cc99">Solution:</span> 
```python

answers_query = """
                SELECT a.id, a.body, a.owner_user_id
                FROM `bigquery-public-data.stackoverflow.posts_questions` AS q 
                INNER JOIN `bigquery-public-data.stackoverflow.posts_answers` AS a
                    ON q.id = a.parent_id
                WHERE q.tags LIKE '%bigquery%'
                """
                
# Set up the query (cancel the query if it would use too much of 
# your quota, with the limit set to 1 GB)
safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=10**10)
answers_query_job = client.query(answers_query, job_config=safe_config)

# API request - run the query, and return a pandas DataFrame
answers_results = answers_query_job.to_dataframe()

```

### 5) Answer the question
You have the merge you need. But you want a list of users who have answered many questions... which requires more work beyond your previous result.

Write a new query that has a single row for each user who answered at least one question with a tag that includes the string "bigquery". Your results should have two columns:
- `user_id` - contains the `owner_user_id` column from the `posts_answers` table
- `number_of_answers` - contains the number of answers the user has written to "bigquery"-related questions

In [ ]:
# Your code here
bigquery_experts_query = """
                         SELECT a.owner_user_id AS user_id, COUNT(1) AS number_of_answers
                         FROM `bigquery-public-data.stackoverflow.posts_questions` AS q
                         INNER JOIN `bigquery-public-data.stackoverflow.posts_answers` AS a
                             ON q.id = a.parent_Id
                         WHERE q.tags LIKE '%bigquery%'
                         GROUP BY a.owner_user_id
                         """

# Set up the query
safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=10**10)
bigquery_experts_query_job = client.query(bigquery_experts_query, job_config=safe_config) # Your code goes here

# API request - run the query, and return a pandas DataFrame
bigquery_experts_results = bigquery_experts_query_job.to_dataframe() # Your code goes here

# Preview results
print(bigquery_experts_results.head())

# Check your answer
q_5.check()

For a hint or the solution, uncomment the appropriate line below.

In [18]:
q_5.hint()
q_5.solution()

<IPython.core.display.Javascript object>

<span style="color:#3366cc">Hint:</span> Start with `SELECT a.owner_user_id AS user_id, COUNT(1) AS number_of_answers`

<IPython.core.display.Javascript object>

<span style="color:#33cc99">Solution:</span> 
```python

bigquery_experts_query = """
                         SELECT a.owner_user_id AS user_id, COUNT(1) AS number_of_answers
                         FROM `bigquery-public-data.stackoverflow.posts_questions` AS q
                         INNER JOIN `bigquery-public-data.stackoverflow.posts_answers` AS a
                             ON q.id = a.parent_Id
                         WHERE q.tags LIKE '%bigquery%'
                         GROUP BY a.owner_user_id
                         """

# Set up the query (cancel the query if it would use too much of 
# your quota, with the limit set to 1 GB)
safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=10**10)
bigquery_experts_query_job = client.query(bigquery_experts_query, job_config=safe_config)

# API request - run the query, and return a pandas DataFrame
bigquery_experts_results = bigquery_experts_query_job.to_dataframe()

```

### 6) Building a more generally useful service

How could you convert what you've done to a general function a website could call on the backend to get experts on any topic?  

Think about it and then check the solution below.

In [ ]:
# Check your answer (Run this code cell to receive credit!)
q_6.solution()

# Congratulations!

You know all the key components to use BigQuery and SQL effectively. Your SQL skills are sufficient to unlock many of the world's largest datasets.

Want to go play with your new powers?  Kaggle has BigQuery datasets available [here](https://www.kaggle.com/datasets?fileType=bigQuery).

---




*Have questions or comments? Visit the [Learn Discussion forum](https://www.kaggle.com/learn-forum/161314) to chat with other Learners.*